# Azure Agent

## Microsoft Agent Framework를 사용한 AI 에이전트 개발 워크샵

이 워크샵에서는 Microsoft Agent Framework와 Azure OpenAI 서비스를 사용하여 다양한 AI 에이전트를 구축하는 방법을 학습합니다.

### 워크샵 목표
- Azure OpenAI 클라이언트(Chat Client, Responses Client, Assistants Client) 이해
- 함수 도구(Function Tools) 및 코드 인터프리터 사용법 학습
- 스레드 관리 및 멀티모달 기능 활용
- 실제 에이전트 구축을 통한 실습

### 필수 요구사항
- Azure OpenAI 서비스 인스턴스
- Python 3.12 이상
- 필요한 패키지 설치 (uv 사용 권장)
- Azure CLI 인증 (`az login`)

## 1. 환경 변수 설정

In [1]:
# Copyright (c) Microsoft. All rights reserved.

from dotenv import load_dotenv
import os

# 환경 변수 로드 (기존 값 덮어쓰기)
load_dotenv(override=True)

# Azure OpenAI 설정 확인
print("Azure OpenAI 설정:")
print(f"Endpoint: {os.environ.get('AZURE_OPENAI_ENDPOINT', 'Not set')}")
print(f"Chat Deployment: {os.environ.get('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME', 'Not set')}")
print(f"Responses Deployment: {os.environ.get('AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME', 'Not set')}")
print(f"API Version: {os.environ.get('AZURE_OPENAI_API_VERSION', 'Not set')}")

# 필요한 환경 변수가 설정되어 있는지 확인
required_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"
]

missing_vars = [var for var in required_vars if not os.environ.get(var)]
if missing_vars:
    print(f"\n⚠️ 다음 환경 변수가 설정되지 않았습니다: {', '.join(missing_vars)}")
    print("Azure OpenAI 서비스 설정을 확인해주세요.")
else:
    print("\n✅ 모든 필수 환경 변수가 설정되었습니다.")

Azure OpenAI 설정:
Endpoint: https://aistudioaiservices343274518006.openai.azure.com
Chat Deployment: gpt-4.1
Responses Deployment: gpt-4.1
API Version: Not set

✅ 모든 필수 환경 변수가 설정되었습니다.


## 2. 필수 패키지 설치

Microsoft Agent Framework 및 Azure 관련 패키지를 설치합니다.

**uv 사용 권장 (Python 3.12):**
```bash
# 프로젝트 의존성 설치
uv sync --dev

# 또는 개별 패키지 설치
uv add agent-framework agent-framework-azure-ai azure-identity python-dotenv pydantic
```

**pip 사용 (대체 방법):**

## 3. 기본 Azure OpenAI Chat Client

Azure OpenAI Chat Client를 사용하여 기본적인 에이전트를 생성하고 날씨 정보를 제공하는 함수 도구를 추가해보겠습니다.

In [2]:
import asyncio
from random import randint
from typing import Annotated

from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from pydantic import Field

# 날씨 정보를 제공하는 함수 도구
def get_weather(
    location: Annotated[str, Field(description="날씨 정보를 가져올 지역명")],
) -> str:
    """지정된 지역의 날씨 정보를 가져옵니다."""
    conditions = ["맑음", "흐림", "비", "폭풍"]
    return f"{location}의 날씨는 {conditions[randint(0, 3)]}이며, 최고 기온은 {randint(10, 30)}°C입니다."

# 비스트리밍 예제
async def non_streaming_example():
    """비스트리밍 응답 예제 (완전한 결과를 한 번에 받기)"""
    print("=== 비스트리밍 응답 예제 ===")

    # Azure Chat Client로 에이전트 생성
    agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
        instructions="당신은 도움이 되는 날씨 에이전트입니다.",
        tools=get_weather,
    )

    query = "서울의 날씨는 어떤가요?"
    print(f"사용자: {query}")
    result = await agent.run(query)
    print(f"결과: {result}\n")

# 스트리밍 예제
async def streaming_example():
    """스트리밍 응답 예제 (결과가 생성되는 대로 받기)"""
    print("=== 스트리밍 응답 예제 ===")

    # Azure Chat Client로 에이전트 생성
    agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
        instructions="당신은 도움이 되는 날씨 에이전트입니다.",
        tools=get_weather,
    )

    query = "부산의 날씨는 어떤가요?"
    print(f"사용자: {query}")
    print("에이전트: ", end="", flush=True)
    async for chunk in agent.run_stream(query):
        if chunk.text:
            print(chunk.text, end="", flush=True)
    print("\n")

# 예제 실행
await non_streaming_example()
await streaming_example()

=== 비스트리밍 응답 예제 ===
사용자: 서울의 날씨는 어떤가요?
결과: 서울의 날씨는 맑고, 최고 기온은 14°C입니다. 외출하시기에 좋은 날씨입니다!

=== 스트리밍 응답 예제 ===
사용자: 부산의 날씨는 어떤가요?
에이전트: 현재 부산의 날씨는 맑으며, 최고 기온은 27°C입니다. 야외 활동하시기에 좋은 날씨입니다!



## 4. Azure OpenAI Chat Client - 함수 도구 패턴

함수 도구를 다양한 방식으로 구성하는 방법을 학습해보겠습니다.

In [3]:
from datetime import datetime, timezone
from agent_framework import ChatAgent

# 현재 시간을 제공하는 함수 도구
def get_time() -> str:
    """현재 UTC 시간을 가져옵니다."""
    current_time = datetime.now(timezone.utc)
    return f"현재 UTC 시간은 {current_time.strftime('%Y-%m-%d %H:%M:%S')}입니다."

# 1. 에이전트 수준에서 도구 정의
async def tools_on_agent_level():
    """에이전트 생성 시 도구를 정의하는 예제"""
    print("=== 에이전트 수준에서 도구 정의 ===")

    # 에이전트 생성 시 도구 제공
    agent = ChatAgent(
        chat_client=AzureOpenAIChatClient(credential=AzureCliCredential()),
        instructions="당신은 날씨와 시간 정보를 제공할 수 있는 도움이 되는 어시스턴트입니다.",
        tools=[get_weather, get_time],  # 에이전트 생성 시 도구 정의
    )

    # 첫 번째 쿼리 - 에이전트가 날씨 도구 사용
    query1 = "뉴욕의 날씨는 어떤가요?"
    print(f"사용자: {query1}")
    result1 = await agent.run(query1)
    print(f"에이전트: {result1}\n")

    # 두 번째 쿼리 - 에이전트가 시간 도구 사용
    query2 = "현재 UTC 시간은 몇 시인가요?"
    print(f"사용자: {query2}")
    result2 = await agent.run(query2)
    print(f"에이전트: {result2}\n")

# 2. 런 메서드에서 도구 전달
async def tools_on_run_level():
    """런 메서드에서 도구를 전달하는 예제"""
    print("=== 런 메서드에서 도구 전달 ===")

    # 도구 없이 에이전트 생성
    agent = ChatAgent(
        chat_client=AzureOpenAIChatClient(credential=AzureCliCredential()),
        instructions="당신은 도움이 되는 어시스턴트입니다.",
    )

    # 날씨 도구와 함께 첫 번째 쿼리
    query1 = "시애틀의 날씨는 어떤가요?"
    print(f"사용자: {query1}")
    result1 = await agent.run(query1, tools=[get_weather])
    print(f"에이전트: {result1}\n")

    # 시간 도구와 함께 두 번째 쿼리
    query2 = "현재 UTC 시간은 몇 시인가요?"
    print(f"사용자: {query2}")
    result2 = await agent.run(query2, tools=[get_time])
    print(f"에이전트: {result2}\n")

# 예제 실행
await tools_on_agent_level()
await tools_on_run_level()

=== 에이전트 수준에서 도구 정의 ===
사용자: 뉴욕의 날씨는 어떤가요?
에이전트: 뉴욕의 현재 날씨는 폭풍이며, 최고 기온은 21°C입니다. 외출 시 주의하시는 것이 좋겠습니다.

사용자: 현재 UTC 시간은 몇 시인가요?
에이전트: 현재 UTC 시간은 2025년 11월 3일 01시 54분 53초입니다.

=== 런 메서드에서 도구 전달 ===
사용자: 시애틀의 날씨는 어떤가요?
에이전트: 시애틀의 현재 날씨는 비가 내리고 있으며, 최고 기온은 22°C입니다. 외출 시 우산을 챙기시는 것이 좋겠습니다!

사용자: 현재 UTC 시간은 몇 시인가요?
에이전트: 현재 UTC 시간은 2025년 11월 3일 01시 54분 55초입니다.



## 5. Azure OpenAI Chat Client - 스레드 관리

스레드를 사용하여 대화 컨텍스트를 유지하는 방법을 학습해보겠습니다.

In [4]:
from agent_framework import AgentThread

# 1. 자동 스레드 생성 예제
async def automatic_thread_example():
    """자동 스레드 생성 예제"""
    print("=== 자동 스레드 생성 예제 ===")

    agent = ChatAgent(
        chat_client=AzureOpenAIChatClient(credential=AzureCliCredential()),
        instructions="당신은 도움이 되는 날씨 에이전트입니다.",
        tools=get_weather,
    )

    # 첫 번째 대화 - 스레드가 제공되지 않으면 자동으로 생성됨
    query1 = "서울의 날씨는 어떤가요?"
    print(f"사용자: {query1}")
    result1 = await agent.run(query1)
    print(f"에이전트: {result1.text}")

    # 두 번째 대화 - 여전히 스레드가 제공되지 않으면 새로운 스레드 생성
    query2 = "제가 마지막으로 물어본 도시는 어디인가요?"
    print(f"\n사용자: {query2}")
    result2 = await agent.run(query2)
    print(f"에이전트: {result2.text}")
    print("참고: 각 호출은 별도의 스레드를 생성하므로 에이전트가 이전 컨텍스트를 기억하지 못합니다.\n")

# 2. 스레드 지속성 예제
async def thread_persistence_example():
    """스레드 지속성을 사용한 컨텍스트 유지 예제"""
    print("=== 스레드 지속성 예제 ===")
    print("동일한 스레드를 여러 대화에 사용하여 컨텍스트를 유지합니다.\n")

    agent = ChatAgent(
        chat_client=AzureOpenAIChatClient(credential=AzureCliCredential()),
        instructions="당신은 도움이 되는 날씨 에이전트입니다.",
        tools=get_weather,
    )

    # 재사용할 새 스레드 생성
    thread = agent.get_new_thread()

    # 첫 번째 대화
    query1 = "도쿄의 날씨는 어떤가요?"
    print(f"사용자: {query1}")
    result1 = await agent.run(query1, thread=thread)
    print(f"에이전트: {result1.text}")

    # 동일한 스레드를 사용한 두 번째 대화 - 컨텍스트 유지
    query2 = "런던은 어떤가요?"
    print(f"\n사용자: {query2}")
    result2 = await agent.run(query2, thread=thread)
    print(f"에이전트: {result2.text}")

    # 세 번째 대화 - 에이전트가 이전 도시들을 기억해야 함
    query3 = "제가 물어본 도시들 중 어느 곳의 날씨가 더 좋나요?"
    print(f"\n사용자: {query3}")
    result3 = await agent.run(query3, thread=thread)
    print(f"에이전트: {result3.text}")
    print("참고: 에이전트가 동일한 스레드의 이전 메시지를 기억합니다.\n")

# 예제 실행
await automatic_thread_example()
await thread_persistence_example()

=== 자동 스레드 생성 예제 ===
사용자: 서울의 날씨는 어떤가요?
에이전트: 서울의 현재 날씨는 흐림이며, 최고 기온은 24도입니다.

사용자: 제가 마지막으로 물어본 도시는 어디인가요?
에이전트: 아직 마지막으로 물어보신 도시가 없습니다. 원하시는 도시의 이름을 말씀해 주시면 그 도시의 날씨 정보를 제공해 드릴 수 있습니다!
참고: 각 호출은 별도의 스레드를 생성하므로 에이전트가 이전 컨텍스트를 기억하지 못합니다.

=== 스레드 지속성 예제 ===
동일한 스레드를 여러 대화에 사용하여 컨텍스트를 유지합니다.

사용자: 도쿄의 날씨는 어떤가요?
에이전트: 도쿄의 현재 날씨는 맑고, 최고 기온은 22°C입니다.

사용자: 런던은 어떤가요?
에이전트: 런던의 현재 날씨는 폭풍이며, 최고 기온은 17°C입니다.

사용자: 제가 물어본 도시들 중 어느 곳의 날씨가 더 좋나요?
에이전트: 도쿄와 런던 중에서는 도쿄의 날씨가 더 좋다고 할 수 있습니다.

도쿄는 맑고, 기온이 22°C로 쾌적한 반면,  
런던은 폭풍이 불고 있고, 기온이 17°C로 다소 쌀쌀합니다.

따라서 날씨면에서는 도쿄가 런던보다 더 좋은 조건입니다.
참고: 에이전트가 동일한 스레드의 이전 메시지를 기억합니다.



## 6. Azure OpenAI Responses Client

구조화된 응답 생성을 위한 Azure OpenAI Responses Client를 사용해보겠습니다.

In [5]:
from agent_framework.azure import AzureOpenAIResponsesClient

# Azure OpenAI Responses Client 기본 사용법
async def responses_client_basic():
    """Azure OpenAI Responses Client 기본 예제"""
    print("=== Azure OpenAI Responses Client 기본 예제 ===")

    # Responses Client로 에이전트 생성
    agent = AzureOpenAIResponsesClient(credential=AzureCliCredential()).create_agent(
        instructions="당신은 도움이 되는 날씨 에이전트입니다.",
        tools=get_weather,
    )

    query = "부산의 날씨는 어떤가요?"
    print(f"사용자: {query}")
    result = await agent.run(query)
    print(f"결과: {result}\n")

# 명시적 설정을 사용한 Responses Client
async def responses_client_explicit_settings():
    """명시적 설정을 사용한 Azure OpenAI Responses Client"""
    print("=== 명시적 설정을 사용한 Responses Client ===")

    # 환경 변수 대신 명시적 설정 사용
    agent = AzureOpenAIResponsesClient(
        deployment_name=os.environ.get("AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME", 
                                       os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")),
        endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        credential=AzureCliCredential(),
    ).create_agent(
        instructions="당신은 도움이 되는 날씨 에이전트입니다.",
        tools=get_weather,
    )

    result = await agent.run("대구의 날씨는 어떤가요?")
    print(f"결과: {result}\n")

# 함수 도구와 함께 사용
async def responses_client_with_tools():
    """함수 도구와 함께 사용하는 Responses Client"""
    print("=== 함수 도구와 함께 사용하는 Responses Client ===")

    agent = ChatAgent(
        chat_client=AzureOpenAIResponsesClient(credential=AzureCliCredential()),
        instructions="당신은 날씨와 시간 정보를 제공할 수 있는 도움이 되는 어시스턴트입니다.",
        tools=[get_weather, get_time],
    )

    # 여러 도구를 사용할 수 있는 쿼리
    query = "광주의 날씨와 현재 UTC 시간을 알려주세요."
    print(f"사용자: {query}")
    result = await agent.run(query)
    print(f"에이전트: {result}\n")

# 예제 실행
await responses_client_basic()
await responses_client_explicit_settings()
await responses_client_with_tools()

=== Azure OpenAI Responses Client 기본 예제 ===
사용자: 부산의 날씨는 어떤가요?
결과: 오늘 부산의 날씨는 맑으며, 최고 기온은 24°C입니다. 좋은 날씨를 즐기시기 바랍니다!

=== 명시적 설정을 사용한 Responses Client ===
결과: 대구의 현재 날씨는 비가 내리고 있으며, 최고 기온은 18°C입니다. 외출 시 우산을 챙기시고, 비에 대비하시기 바랍니다.

=== 함수 도구와 함께 사용하는 Responses Client ===
사용자: 광주의 날씨와 현재 UTC 시간을 알려주세요.
에이전트: - 광주의 날씨는 폭풍이며, 최고 기온은 26°C입니다.
- 현재 UTC 시간은 2025-11-03 01:55:11입니다.



## 7. Azure OpenAI Responses Client - 코드 인터프리터

Python 코드를 실행할 수 있는 코드 인터프리터 기능을 사용해보겠습니다.

In [7]:
from agent_framework import HostedCodeInterpreterTool, ChatResponse
from openai.types.responses.response import Response as OpenAIResponse
from openai.types.responses.response_code_interpreter_tool_call import ResponseCodeInterpreterToolCall

async def code_interpreter_example():
    """코드 인터프리터를 사용한 수학 문제 해결 예제"""
    print("=== Azure OpenAI Responses Agent와 코드 인터프리터 예제 ===")

    try:
        # 코드 인터프리터 도구가 포함된 에이전트 생성
        agent = ChatAgent(
            chat_client=AzureOpenAIResponsesClient(credential=AzureCliCredential()),
            instructions="당신은 코드를 작성하고 실행하여 문제를 해결할 수 있는 도움이 되는 어시스턴트입니다.",
            tools=HostedCodeInterpreterTool(),
        )

        query = "코드를 사용하여 100의 팩토리얼을 계산해주세요."
        print(f"사용자: {query}")
        result = await agent.run(query)
        print(f"결과: {result}\n")

        # 생성된 코드 추출 (가능한 경우)
        if (
            isinstance(result.raw_representation, ChatResponse)
            and isinstance(result.raw_representation.raw_representation, OpenAIResponse)
            and len(result.raw_representation.raw_representation.output) > 0
            and isinstance(result.raw_representation.raw_representation.output[0], ResponseCodeInterpreterToolCall)
        ):
            generated_code = result.raw_representation.raw_representation.output[0].code
            print(f"생성된 코드:\n{generated_code}")
    except Exception as e:
        print(f"⚠️ 코드 인터프리터를 사용할 수 없습니다: {str(e)}")
        print("이 리전에서는 코드 인터프리터가 지원되지 않습니다.")
        print("대신 일반 함수 도구를 사용한 예제를 확인하세요.\n")

# 더 복잡한 계산 예제
async def complex_calculation_example():
    """복잡한 계산을 위한 코드 인터프리터 예제"""
    print("=== 복잡한 계산 예제 ===")

    try:
        agent = ChatAgent(
            chat_client=AzureOpenAIResponsesClient(credential=AzureCliCredential()),
            instructions="당신은 수학적 분석과 계산을 수행할 수 있는 전문가입니다.",
            tools=HostedCodeInterpreterTool(),
        )

        query = "1부터 1000까지의 소수를 찾고, 그 개수와 평균을 계산해주세요."
        print(f"사용자: {query}")
        result = await agent.run(query)
        print(f"결과: {result}\n")
    except Exception as e:
        print(f"⚠️ 코드 인터프리터를 사용할 수 없습니다: {str(e)}")
        print("이 리전에서는 코드 인터프리터가 지원되지 않습니다.\n")

# 데이터 분석 예제
async def data_analysis_example():
    """간단한 데이터 분석 예제"""
    print("=== 데이터 분석 예제 ===")

    try:
        agent = ChatAgent(
            chat_client=AzureOpenAIResponsesClient(credential=AzureCliCredential()),
            instructions="당신은 데이터 분석을 수행할 수 있는 전문가입니다.",
            tools=HostedCodeInterpreterTool(),
        )

        query = """다음 판매 데이터를 분석해주세요:
        1월: 100, 2월: 120, 3월: 95, 4월: 140, 5월: 160, 6월: 180
        월별 성장률과 평균 매출을 계산하고 시각화해주세요."""
        
        print(f"사용자: {query}")
        result = await agent.run(query)
        print(f"결과: {result}\n")
    except Exception as e:
        print(f"⚠️ 코드 인터프리터를 사용할 수 없습니다: {str(e)}")
        print("이 리전에서는 코드 인터프리터가 지원되지 않습니다.\n")

# 코드 인터프리터 대신 사용할 수 있는 일반 함수 도구 예제
def calculate_factorial(n: Annotated[int, Field(description="팩토리얼을 계산할 숫자")]) -> str:
    """주어진 숫자의 팩토리얼을 계산합니다."""
    import math
    result = math.factorial(n)
    return f"{n}의 팩토리얼은 {result}입니다."

async def alternative_calculation_example():
    """코드 인터프리터 대신 함수 도구를 사용한 예제"""
    print("=== 함수 도구를 사용한 계산 예제 (코드 인터프리터 대안) ===")
    
    agent = ChatAgent(
        chat_client=AzureOpenAIResponsesClient(credential=AzureCliCredential()),
        instructions="당신은 수학적 계산을 도와주는 어시스턴트입니다.",
        tools=calculate_factorial,
    )

    query = "100의 팩토리얼을 계산해주세요."
    print(f"사용자: {query}")
    result = await agent.run(query)
    print(f"결과: {result}\n")

# 예제 실행
await code_interpreter_example()
await complex_calculation_example()
await data_analysis_example()
print("\n코드 인터프리터가 지원되지 않는 경우, 아래 대안을 사용할 수 있습니다:")
await alternative_calculation_example()

=== Azure OpenAI Responses Agent와 코드 인터프리터 예제 ===
사용자: 코드를 사용하여 100의 팩토리얼을 계산해주세요.
⚠️ 코드 인터프리터를 사용할 수 없습니다: <class 'agent_framework.azure._responses_client.AzureOpenAIResponsesClient'> service failed to complete the prompt: Error code: 400 - {'error': {'message': 'Code interpreter tool cannot be used in this region.', 'type': 'invalid_request_error', 'param': 'tools', 'code': 'unsupported_parameter'}}
이 리전에서는 코드 인터프리터가 지원되지 않습니다.
대신 일반 함수 도구를 사용한 예제를 확인하세요.

=== 복잡한 계산 예제 ===
사용자: 1부터 1000까지의 소수를 찾고, 그 개수와 평균을 계산해주세요.
⚠️ 코드 인터프리터를 사용할 수 없습니다: <class 'agent_framework.azure._responses_client.AzureOpenAIResponsesClient'> service failed to complete the prompt: Error code: 400 - {'error': {'message': 'Code interpreter tool cannot be used in this region.', 'type': 'invalid_request_error', 'param': 'tools', 'code': 'unsupported_parameter'}}
이 리전에서는 코드 인터프리터가 지원되지 않습니다.
대신 일반 함수 도구를 사용한 예제를 확인하세요.

=== 복잡한 계산 예제 ===
사용자: 1부터 1000까지의 소수를 찾고, 그 개수와 평균을 계산해주세요.
⚠️ 코드 인터프리터를 사용할 수 없습니다: <class 'ag

## 8. Azure OpenAI Responses Client - 이미지 분석

비전 모델을 사용한 이미지 분석 기능을 사용해보겠습니다.

In [9]:
from agent_framework import ChatMessage, TextContent, UriContent
from IPython.display import display, Image as IPImage, HTML
import requests
from io import BytesIO

def display_image_from_url(url: str, max_width: int = 600):
    """URL에서 이미지를 가져와 노트북에 표시합니다."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        img = IPImage(response.content)
        display(HTML(f'<img src="{url}" style="max-width:{max_width}px;"/>'))
    except Exception as e:
        print(f"이미지를 표시할 수 없습니다: {e}")

async def image_analysis_example():
    """이미지 분석 예제"""
    print("=== Azure Responses Agent와 이미지 분석 ===\n")

    # 비전 기능이 있는 Azure Responses 에이전트 생성
    agent = AzureOpenAIResponsesClient(credential=AzureCliCredential()).create_agent(
        name="VisionAgent",
        instructions="당신은 이미지를 분석할 수 있는 도움이 되는 에이전트입니다.",
    )

    # 이미지 URL
    image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
    
    # 이미지 표시
    print("분석할 이미지:")
    display_image_from_url(image_url)
    print()

    # 텍스트와 이미지 콘텐츠를 모두 포함하는 메시지 생성
    user_message = ChatMessage(
        role="user",
        contents=[
            TextContent(text="이 이미지에서 무엇을 보시나요?"),
            UriContent(
                uri=image_url,
                media_type="image/jpeg",
            ),
        ],
    )

    # 에이전트의 응답 받기
    print("사용자: 이 이미지에서 무엇을 보시나요?")
    result = await agent.run(user_message)
    print(f"에이전트: {result.text}")
    print()

# 여러 이미지 분석 예제
async def multiple_images_example():
    """여러 이미지를 비교 분석하는 예제"""
    print("=== 여러 이미지 비교 분석 ===\n")

    agent = AzureOpenAIResponsesClient(credential=AzureCliCredential()).create_agent(
        name="VisionAgent",
        instructions="당신은 이미지를 비교하고 분석할 수 있는 전문가입니다.",
    )

    # 자연 풍경 이미지 URI
    nature_image = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
    
    # 이미지 표시
    print("분석할 이미지:")
    display_image_from_url(nature_image)
    print()
    
    user_message = ChatMessage(
        role="user",
        contents=[
            TextContent(text="이 이미지의 색상, 구성, 분위기를 자세히 분석해주세요."),
            UriContent(
                uri=nature_image,
                media_type="image/jpeg",
            ),
        ],
    )

    print("사용자: 이 이미지의 색상, 구성, 분위기를 자세히 분석해주세요.")
    result = await agent.run(user_message)
    print(f"에이전트: {result.text}")
    print()

# 이미지 기반 질문 답변 예제
async def image_qa_example():
    """이미지 기반 질문 답변 예제"""
    print("=== 이미지 기반 질문 답변 ===\n")

    agent = AzureOpenAIResponsesClient(credential=AzureCliCredential()).create_agent(
        name="VisionQAAgent",
        instructions="당신은 이미지를 보고 구체적인 질문에 답할 수 있는 전문가입니다.",
    )

    image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
    
    # 이미지 표시
    print("분석할 이미지:")
    display_image_from_url(image_url)
    print()

    user_message = ChatMessage(
        role="user",
        contents=[
            TextContent(text="이 이미지에 사람이 있나요? 날씨는 어때 보이나요? 이곳이 어떤 종류의 장소인지 설명해주세요."),
            UriContent(
                uri=image_url,
                media_type="image/jpeg",
            ),
        ],
    )

    print("사용자: 이 이미지에 사람이 있나요? 날씨는 어때 보이나요? 이곳이 어떤 종류의 장소인지 설명해주세요.")
    result = await agent.run(user_message)
    print(f"에이전트: {result.text}")
    print()

# 여러 이미지 비교 예제 (추가)
async def compare_multiple_images_example():
    """여러 이미지를 나란히 보여주고 비교 분석"""
    print("=== 여러 이미지 비교 분석 (Side by Side) ===\n")

    agent = AzureOpenAIResponsesClient(credential=AzureCliCredential()).create_agent(
        name="VisionAgent",
        instructions="당신은 여러 이미지를 비교하고 분석할 수 있는 전문가입니다.",
    )

    # 두 개의 다른 이미지 URL
    image1_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/800px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
    image2_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3f/Fronalpstock_big.jpg/800px-Fronalpstock_big.jpg"
    
    # 이미지들을 나란히 표시
    print("비교할 이미지들:")
    display(HTML(f'''
        <div style="display: flex; gap: 20px; justify-content: center;">
            <div>
                <p><b>이미지 1</b></p>
                <img src="{image1_url}" style="max-width:400px;"/>
            </div>
            <div>
                <p><b>이미지 2</b></p>
                <img src="{image2_url}" style="max-width:400px;"/>
            </div>
        </div>
    '''))
    print()

    user_message = ChatMessage(
        role="user",
        contents=[
            TextContent(text="이 두 이미지를 비교하여 공통점과 차이점을 설명해주세요."),
            UriContent(uri=image1_url, media_type="image/jpeg"),
            UriContent(uri=image2_url, media_type="image/jpeg"),
        ],
    )

    print("사용자: 이 두 이미지를 비교하여 공통점과 차이점을 설명해주세요.")
    result = await agent.run(user_message)
    print(f"에이전트: {result.text}")
    print()

# 예제 실행
await image_analysis_example()
await multiple_images_example()
await image_qa_example()
await compare_multiple_images_example()

=== Azure Responses Agent와 이미지 분석 ===

분석할 이미지:
이미지를 표시할 수 없습니다: 403 Client Error: Forbidden for url: https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg

사용자: 이 이미지에서 무엇을 보시나요?
이미지를 표시할 수 없습니다: 403 Client Error: Forbidden for url: https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg

사용자: 이 이미지에서 무엇을 보시나요?
에이전트: 이 이미지에는 평평한 들판 위에 나무 데크로 이루어진 산책로가 중앙에서 바깥쪽으로 뻗어 있습니다. 산책로 양쪽에는 푸른 풀과 식물들이 무성하게 자라 있고, 멀리에는 나무와 덤불이 보입니다. 하늘은 맑고 푸르며, 곳곳에 흰 구름이 떠 있습니다. 전체적으로 자연 풍경이 평화롭고 밝게 표현되어 있으며, 산책이나 휴식에 적합한 분위기입니다.

=== 여러 이미지 비교 분석 ===

분석할 이미지:
에이전트: 이 이미지에는 평평한 들판 위에 나무 데크로 이루어진 산책로가 중앙에서 바깥쪽으로 뻗어 있습니다. 산책로 양쪽에는 푸른 풀과 식물들이 무성하게 자라 있고, 멀리에는 나무와 덤불이 보입니다. 하늘은 맑고 푸르며, 곳곳에 흰 구름이 떠 있습니다. 전체적으로 자연 풍경이 평화롭고 밝게 표현되어 있으며, 산책이나 휴식에 적합한 분위기입니다.

=== 여러 이미지 비교 분석 ===

분석할 이미지:
이미지를 표시할 수 없습니다: 403 Clien


사용자: 이 두 이미지를 비교하여 공통점과 차이점을 설명해주세요.
에이전트: 두 이미지를 비교하여 공통점과 차이점을 아래와 같이 설명드릴 수 있습니다.

---

### **공통점**
1. **자연 풍경**  
   두 이미지 모두 자연을 담고 있습니다. 초록색이 많이 보이고, 푸른 하늘이 공통적으로 있습니다.

2. **야외 촬영**  
   두 사진 모두 야외에서 찍힌 사진으로, 사람의 손길이 크게 닿지 않은 듯한 모습입니다.

3. **푸른 하늘**  
   하늘이 넓게 펼쳐져 있고 구름이 점점이 떠 있는 모습이 두 사진 모두 보입니다.

---

### **차이점**
1. **지형의 특징**  
   - **첫 번째 이미지**: 평평한 초원(습지 또는 들판)이 주요 배경입니다. 가운데에 나무 데크(보행로)가 보입니다.
   - **두 번째 이미지**: 높은 산과 언덕이 배경입니다. 산악지대의 웅장함이 느껴집니다.

2. **구성 요소**  
   - **첫 번째 이미지**: 주요 요소는 초원, 나무, 그리고 나무로 만들어진 길이 있습니다.
   - **두 번째 이미지**: 주요 요소는 산, 숲, 그리고 멀리 작은 마을이나 초지가 있습니다. 길이나 인공 구조물은 잘 보이지 않습니다.

3. **색감과 분위기**  
   - **첫 번째 이미지**: 좀 더 따뜻하고 밝은 분위기(노란빛의 풀, 파란 하늘), 평화롭고 여유로운 느낌.
   - **두 번째 이미지**: 진한 녹색의 산과 파란 하늘, 자연의 웅장함과 신비로움, 그리고 시원한 느낌.

4. **원근감**  
   - **첫 번째 이미지**: 데크가 원근감을 강조하며 시선이 멀리까지 이어집니다.
   - **두 번째 이미지**: 산의 높이와 넓이가 시야를 압도하고, 원근감은 있지만 사진 전체가 산지로 채워져 있음.

---

### **요약**
두 이미지는 모두 자연을 담고 있고, 푸른 하늘과 녹색 식물의 조화가 특징입니다. 그러나 첫 번째는 평야와 나무 데크가 중심이며, 두 번째는 산악지대와 그 

## 9. Azure OpenAI Assistants - 기본 사용법

Azure OpenAI Assistants의 기본 사용법과 자동 어시스턴트 생명주기 관리를 학습해보겠습니다.

In [10]:
from agent_framework.azure import AzureOpenAIAssistantsClient

# 비스트리밍 예제
async def assistants_non_streaming():
    """Azure OpenAI Assistants 비스트리밍 예제"""
    print("=== Azure OpenAI Assistants 비스트리밍 예제 ===")

    # 어시스턴트 ID가 제공되지 않으므로 어시스턴트가 자동으로 생성되고
    # 응답을 받은 후 삭제됩니다
    async with AzureOpenAIAssistantsClient(credential=AzureCliCredential()).create_agent(
        instructions="당신은 도움이 되는 날씨 에이전트입니다.",
        tools=get_weather,
    ) as agent:
        query = "서울의 날씨는 어떤가요?"
        print(f"사용자: {query}")
        result = await agent.run(query)
        print(f"에이전트: {result}\n")

# 스트리밍 예제
async def assistants_streaming():
    """Azure OpenAI Assistants 스트리밍 예제"""
    print("=== Azure OpenAI Assistants 스트리밍 예제 ===")

    # 어시스턴트 ID가 제공되지 않으므로 어시스턴트가 자동으로 생성되고
    # 응답을 받은 후 삭제됩니다
    async with AzureOpenAIAssistantsClient(credential=AzureCliCredential()).create_agent(
        instructions="당신은 도움이 되는 날씨 에이전트입니다.",
        tools=get_weather,
    ) as agent:
        query = "부산의 날씨는 어떤가요?"
        print(f"사용자: {query}")
        print("에이전트: ", end="", flush=True)
        async for chunk in agent.run_stream(query):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        print("\n")

# 명시적 설정을 사용한 예제
async def assistants_explicit_settings():
    """명시적 설정을 사용한 Azure OpenAI Assistants 예제"""
    print("=== 명시적 설정을 사용한 Assistants 예제 ===")

    # 환경 변수 대신 명시적 설정 사용
    async with AzureOpenAIAssistantsClient(
        endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        deployment_name=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
        credential=AzureCliCredential(),
    ).create_agent(
        instructions="당신은 도움이 되는 날씨 에이전트입니다.",
        tools=get_weather,
    ) as agent:
        result = await agent.run("대구의 날씨는 어떤가요?")
        print(f"결과: {result}\n")

# 예제 실행
await assistants_non_streaming()
await assistants_streaming()
await assistants_explicit_settings()

=== Azure OpenAI Assistants 비스트리밍 예제 ===
사용자: 서울의 날씨는 어떤가요?


[2025-11-03 10:59:10 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


에이전트: 서울의 날씨는 맑고, 최고 기온은 25°C입니다. 야외 활동하기에 좋은 날씨입니다!

=== Azure OpenAI Assistants 스트리밍 예제 ===
사용자: 부산의 날씨는 어떤가요?
에이전트: 

[2025-11-03 10:59:14 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:704 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


부산산의 오늘의 오늘 날씨 날씨는 흐는 흐림림이며 최고 기이며 최고 기온은온은 24 24°C입니다°C입니다. 외. 외출 시출 시 흐린 흐린 하 하늘을늘을 참고하시기 참고하시기 바랍니다! 바랍니다!



=== 명시적 설정을 사용한 Assistants 예제 ===
=== 명시적 설정을 사용한 Assistants 예제 ===


[2025-11-03 10:59:18 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


결과: 대구의 오늘 날씨는 흐림이며, 최고 기온은 19°C입니다. 외출 시에는 흐린 하늘을 참고하시기 바랍니다.



## 10. Azure OpenAI Assistants - 함수 도구

Azure OpenAI Assistants에서 함수 도구를 사용하는 다양한 패턴을 학습해보겠습니다.

In [11]:
# 에이전트 수준에서 도구 정의
async def assistants_tools_on_agent():
    """Azure OpenAI Assistants에서 에이전트 수준 도구 정의"""
    print("=== Assistants 에이전트 수준 도구 정의 ===")

    # 에이전트 생성 시 도구 제공
    async with ChatAgent(
        chat_client=AzureOpenAIAssistantsClient(credential=AzureCliCredential()),
        instructions="당신은 날씨와 시간 정보를 제공할 수 있는 도움이 되는 어시스턴트입니다.",
        tools=[get_weather, get_time],  # 에이전트 생성 시 도구 정의
    ) as agent:
        # 첫 번째 쿼리 - 에이전트가 날씨 도구 사용
        query1 = "뉴욕의 날씨는 어떤가요?"
        print(f"사용자: {query1}")
        result1 = await agent.run(query1)
        print(f"에이전트: {result1}\n")

        # 두 번째 쿼리 - 에이전트가 시간 도구 사용
        query2 = "현재 UTC 시간은 몇 시인가요?"
        print(f"사용자: {query2}")
        result2 = await agent.run(query2)
        print(f"에이전트: {result2}\n")

        # 세 번째 쿼리 - 필요시 두 도구 모두 사용
        query3 = "런던의 날씨와 현재 UTC 시간을 알려주세요."
        print(f"사용자: {query3}")
        result3 = await agent.run(query3)
        print(f"에이전트: {result3}\n")

# 런 메서드에서 도구 전달
async def assistants_tools_on_run():
    """Azure OpenAI Assistants에서 런 메서드 도구 전달"""
    print("=== Assistants 런 메서드 도구 전달 ===")

    # 도구 없이 에이전트 생성
    async with ChatAgent(
        chat_client=AzureOpenAIAssistantsClient(credential=AzureCliCredential()),
        instructions="당신은 도움이 되는 어시스턴트입니다.",
    ) as agent:
        # 날씨 도구와 함께 첫 번째 쿼리
        query1 = "시애틀의 날씨는 어떤가요?"
        print(f"사용자: {query1}")
        result1 = await agent.run(query1, tools=[get_weather])
        print(f"에이전트: {result1}\n")

        # 시간 도구와 함께 두 번째 쿼리
        query2 = "현재 UTC 시간은 몇 시인가요?"
        print(f"사용자: {query2}")
        result2 = await agent.run(query2, tools=[get_time])
        print(f"에이전트: {result2}\n")

        # 여러 도구와 함께 세 번째 쿼리
        query3 = "시카고의 날씨와 현재 UTC 시간을 알려주세요."
        print(f"사용자: {query3}")
        result3 = await agent.run(query3, tools=[get_weather, get_time])
        print(f"에이전트: {result3}\n")

# 스레드 지속성과 함께 사용
async def assistants_with_threads():
    """Azure OpenAI Assistants의 스레드 지속성 예제"""
    print("=== Assistants 스레드 지속성 예제 ===")

    async with ChatAgent(
        chat_client=AzureOpenAIAssistantsClient(credential=AzureCliCredential()),
        instructions="당신은 도움이 되는 날씨 에이전트입니다.",
        tools=get_weather,
    ) as agent:
        # 재사용할 새 스레드 생성
        thread = agent.get_new_thread()

        # 첫 번째 대화
        query1 = "도쿄의 날씨는 어떤가요?"
        print(f"사용자: {query1}")
        result1 = await agent.run(query1, thread=thread)
        print(f"에이전트: {result1.text}")

        # 동일한 스레드를 사용한 두 번째 대화 - 컨텍스트 유지
        query2 = "런던은 어떤가요?"
        print(f"\n사용자: {query2}")
        result2 = await agent.run(query2, thread=thread)
        print(f"에이전트: {result2.text}")

        # 세 번째 대화 - 에이전트가 이전 도시들을 기억해야 함
        query3 = "제가 물어본 도시들 중 어느 곳의 날씨가 더 좋나요?"
        print(f"\n사용자: {query3}")
        result3 = await agent.run(query3, thread=thread)
        print(f"에이전트: {result3.text}")
        print("참고: 에이전트가 동일한 스레드의 이전 메시지를 기억합니다.\n")

# 예제 실행
await assistants_tools_on_agent()
await assistants_tools_on_run()
await assistants_with_threads()

=== Assistants 에이전트 수준 도구 정의 ===
사용자: 뉴욕의 날씨는 어떤가요?


[2025-11-03 10:59:28 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


에이전트: 뉴욕의 현재 날씨는 흐림이고, 최고 기온은 22°C입니다.

사용자: 현재 UTC 시간은 몇 시인가요?


[2025-11-03 10:59:32 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


에이전트: 현재 UTC 시간은 2025년 11월 3일 01시 59분 32초입니다.

사용자: 런던의 날씨와 현재 UTC 시간을 알려주세요.


...

에이전트: 시카고의 날씨는 폭풍이며, 최고 기온은 21°C입니다.
현재 UTC 시간은 2025-11-03 01:59:45입니다.

=== Assistants 스레드 지속성 예제 ===
사용자: 도쿄의 날씨는 어떤가요?


[2025-11-03 10:59:49 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.
[2025-11-03 10:59:51 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.
[2025-11-03 10:59:51 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


에이전트: 도쿄의 현재 날씨는 흐림이고, 최고 기온은 16°C입니다. 외출 시에는 흐린 하늘과 약간 선선한 온도에 대비해 주세요!

사용자: 런던은 어떤가요?


[2025-11-03 10:59:52 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.
[2025-11-03 10:59:54 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.
[2025-11-03 10:59:54 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


에이전트: 런던의 현재 날씨는 맑고, 최고 기온은 19°C입니다. 날씨가 좋아서 야외 활동하기에 좋은 날입니다!

사용자: 제가 물어본 도시들 중 어느 곳의 날씨가 더 좋나요?
에이전트: 두 도시의 날씨를 비교해보면:

- 도쿄: 흐림, 최고 기온 16°C
- 런던: 맑음, 최고 기온 19°C

일반적으로 맑은 날씨가 흐린 날씨보다 더 쾌적하다고 평가받기 때문에, 오늘은 런던의 날씨가 도쿄보다 더 좋다고 할 수 있습니다. 야외 활동이나 산책을 하기에는 런던이 더 적합한 날씨입니다!
참고: 에이전트가 동일한 스레드의 이전 메시지를 기억합니다.

에이전트: 두 도시의 날씨를 비교해보면:

- 도쿄: 흐림, 최고 기온 16°C
- 런던: 맑음, 최고 기온 19°C

일반적으로 맑은 날씨가 흐린 날씨보다 더 쾌적하다고 평가받기 때문에, 오늘은 런던의 날씨가 도쿄보다 더 좋다고 할 수 있습니다. 야외 활동이나 산책을 하기에는 런던이 더 적합한 날씨입니다!
참고: 에이전트가 동일한 스레드의 이전 메시지를 기억합니다.



## 11. Azure OpenAI Assistants - 코드 인터프리터

Azure OpenAI Assistants와 HostedCodeInterpreterTool을 결합하여 고급 코드 실행 기능을 사용해보겠습니다.

In [12]:
from agent_framework import AgentRunResponseUpdate, ChatResponseUpdate
from openai.types.beta.threads.runs import (
    CodeInterpreterToolCallDelta,
    RunStepDelta,
    RunStepDeltaEvent,
    ToolCallDeltaObject,
)
from openai.types.beta.threads.runs.code_interpreter_tool_call_delta import CodeInterpreter

def get_code_interpreter_chunk(chunk: AgentRunResponseUpdate) -> str | None:
    """코드 인터프리터 데이터에 접근하는 헬퍼 메서드"""
    if (
        isinstance(chunk.raw_representation, ChatResponseUpdate)
        and isinstance(chunk.raw_representation.raw_representation, RunStepDeltaEvent)
        and isinstance(chunk.raw_representation.raw_representation.delta, RunStepDelta)
        and isinstance(chunk.raw_representation.raw_representation.delta.step_details, ToolCallDeltaObject)
        and chunk.raw_representation.raw_representation.delta.step_details.tool_calls
    ):
        for tool_call in chunk.raw_representation.raw_representation.delta.step_details.tool_calls:
            if (
                isinstance(tool_call, CodeInterpreterToolCallDelta)
                and isinstance(tool_call.code_interpreter, CodeInterpreter)
                and tool_call.code_interpreter.input is not None
            ):
                return tool_call.code_interpreter.input
    return None

async def assistants_code_interpreter():
    """Azure OpenAI Assistants와 코드 인터프리터 사용 예제"""
    print("=== Azure OpenAI Assistants Agent와 코드 인터프리터 예제 ===")

    async with ChatAgent(
        chat_client=AzureOpenAIAssistantsClient(credential=AzureCliCredential()),
        instructions="당신은 코드를 작성하고 실행하여 문제를 해결할 수 있는 도움이 되는 어시스턴트입니다.",
        tools=HostedCodeInterpreterTool(),
    ) as agent:
        query = "현재 날짜와 시간을 가져와주세요."
        print(f"사용자: {query}")
        print("에이전트: ", end="", flush=True)
        generated_code = ""
        async for chunk in agent.run_stream(query):
            if chunk.text:
                print(chunk.text, end="", flush=True)
            code_interpreter_chunk = get_code_interpreter_chunk(chunk)
            if code_interpreter_chunk is not None:
                generated_code += code_interpreter_chunk

        print(f"\n생성된 코드:\n{generated_code}")

# 수학적 계산 예제
async def assistants_math_calculation():
    """Assistants를 사용한 수학적 계산 예제"""
    print("\n=== Assistants 수학적 계산 예제 ===")

    async with ChatAgent(
        chat_client=AzureOpenAIAssistantsClient(credential=AzureCliCredential()),
        instructions="당신은 복잡한 수학 계산을 수행할 수 있는 전문가입니다.",
        tools=HostedCodeInterpreterTool(),
    ) as agent:
        query = "피보나치 수열의 처음 20개 항을 계산하고, 그 합을 구해주세요."
        print(f"사용자: {query}")
        result = await agent.run(query)
        print(f"에이전트: {result.text}")

# 데이터 처리 예제
async def assistants_data_processing():
    """Assistants를 사용한 데이터 처리 예제"""
    print("\n=== Assistants 데이터 처리 예제 ===")

    async with ChatAgent(
        chat_client=AzureOpenAIAssistantsClient(credential=AzureCliCredential()),
        instructions="당신은 데이터 분석과 시각화를 수행할 수 있는 전문가입니다.",
        tools=HostedCodeInterpreterTool(),
    ) as agent:
        query = """다음 학생들의 성적 데이터를 분석해주세요:
        김철수: 수학 85, 영어 92, 과학 78
        이영희: 수학 94, 영어 88, 과학 91
        박민수: 수학 76, 영어 83, 과학 89
        최지은: 수학 88, 영어 95, 과학 87
        
        각 과목별 평균과 각 학생별 평균을 계산하고, 
        성적이 가장 높은 학생과 과목을 찾아주세요."""
        
        print(f"사용자: {query}")
        result = await agent.run(query)
        print(f"에이전트: {result.text}")

# 예제 실행
await assistants_code_interpreter()
await assistants_math_calculation()
await assistants_data_processing()

=== Azure OpenAI Assistants Agent와 코드 인터프리터 예제 ===
사용자: 현재 날짜와 시간을 가져와주세요.
에이전트: 현재 날짜현재 날짜와 시간은와 시간은  20252025년 년 11월11월 3 3일 일 02시02시 00 00분 분 00초00초입니다입니다..
생성된 코드:
from datetime import datetime

# 현재 날짜와 시간을 가져옴
now = datetime.now()
now

생성된 코드:
from datetime import datetime

# 현재 날짜와 시간을 가져옴
now = datetime.now()
now

=== Assistants 수학적 계산 예제 ===
사용자: 피보나치 수열의 처음 20개 항을 계산하고, 그 합을 구해주세요.

=== Assistants 수학적 계산 예제 ===
사용자: 피보나치 수열의 처음 20개 항을 계산하고, 그 합을 구해주세요.
에이전트: 피보나치 수열의 처음 20개 항과 그 합을 구해드리겠습니다.

피보나치 수열의 항 \( F_n \)은 다음과 같이 정의됩니다:
- \( F_1 = 1 \)
- \( F_2 = 1 \)
- \( F_n = F_{n-1} + F_{n-2} \) (for \( n \geq 3 \))

계산해서 보여드릴게요.피보나치 수열의 처음 20개 항은 다음과 같습니다:

1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765

이 20개 항의 합은 **17,710**입니다.

=== Assistants 데이터 처리 예제 ===
사용자: 다음 학생들의 성적 데이터를 분석해주세요:
        김철수: 수학 85, 영어 92, 과학 78
        이영희: 수학 94, 영어 88, 과학 91
        박민수: 수학 76, 영어 83, 과학 89
        최지은: 수학 88, 영어 95, 과학 87

        각 과목별 평균과 각 학생

## 12. 워크샵 실습: 나만의 에이전트 만들기

지금까지 학습한 내용을 바탕으로 여러 기능을 결합한 종합적인 에이전트를 만들어보겠습니다.

In [13]:
# 실습 과제: 종합 에이전트 만들기
# 다음 기능들을 가진 "여행 계획 도우미" 에이전트를 만들어보세요:
# 1. 날씨 정보 제공 (get_weather 함수 사용)
# 2. 현재 시간 제공 (get_time 함수 사용)
# 3. 여행 예산 계산 (새로운 함수 작성)
# 4. 스레드를 사용한 대화 컨텍스트 유지

# 여행 예산을 계산하는 함수
def calculate_travel_budget(
    days: Annotated[int, Field(description="여행 일수")],
    daily_budget: Annotated[float, Field(description="일일 예산 (원)")],
    num_people: Annotated[int, Field(description="여행 인원 수")] = 1,
) -> str:
    """여행 예산을 계산합니다."""
    total_budget = days * daily_budget * num_people
    return f"{num_people}명이 {days}일간 여행할 경우, 총 예산은 {total_budget:,.0f}원입니다."

# 항공권 가격 추정 함수
def estimate_flight_price(
    destination: Annotated[str, Field(description="목적지")],
    departure: Annotated[str, Field(description="출발지")] = "서울",
) -> str:
    """항공권 가격을 추정합니다."""
    prices = {
        "도쿄": 300000,
        "오사카": 280000,
        "방콕": 450000,
        "싱가포르": 550000,
        "파리": 1200000,
        "런던": 1300000,
        "뉴욕": 1400000,
        "시드니": 1100000,
    }
    price = prices.get(destination, 500000)
    return f"{departure}에서 {destination}까지의 왕복 항공권 예상 가격은 약 {price:,.0f}원입니다."

# 여행 계획 도우미 에이전트 생성
async def travel_planning_agent():
    """종합적인 여행 계획 도우미 에이전트"""
    print("=== 여행 계획 도우미 에이전트 ===\n")

    # 모든 도구를 포함한 에이전트 생성
    async with ChatAgent(
        chat_client=AzureOpenAIAssistantsClient(credential=AzureCliCredential()),
        instructions="""당신은 여행 계획을 도와주는 전문 에이전트입니다. 
        다음과 같은 기능을 제공할 수 있습니다:
        1. 목적지의 날씨 정보
        2. 현재 시간 확인
        3. 여행 예산 계산
        4. 항공권 가격 추정
        
        사용자의 질문에 친절하고 자세하게 답변해주세요.""",
        tools=[get_weather, get_time, calculate_travel_budget, estimate_flight_price],
    ) as agent:
        # 대화 컨텍스트를 유지하기 위한 스레드 생성
        thread = agent.get_new_thread()

        # 시나리오 1: 기본 정보 수집
        query1 = "도쿄로 여행을 가려고 하는데, 현재 날씨는 어떤가요?"
        print(f"사용자: {query1}")
        result1 = await agent.run(query1, thread=thread)
        print(f"에이전트: {result1.text}\n")

        # 시나리오 2: 항공권 정보
        query2 = "도쿄까지 항공권은 얼마나 할까요?"
        print(f"사용자: {query2}")
        result2 = await agent.run(query2, thread=thread)
        print(f"에이전트: {result2.text}\n")

        # 시나리오 3: 예산 계산
        query3 = "5일 동안 2명이 가는데, 일일 예산을 15만원으로 잡으면 총 얼마나 필요한가요?"
        print(f"사용자: {query3}")
        result3 = await agent.run(query3, thread=thread)
        print(f"에이전트: {result3.text}\n")

        # 시나리오 4: 종합 상담 (컨텍스트 활용)
        query4 = "지금까지 이야기한 내용을 바탕으로 총 여행 경비를 정리해주세요."
        print(f"사용자: {query4}")
        result4 = await agent.run(query4, thread=thread)
        print(f"에이전트: {result4.text}\n")

        # 시나리오 5: 현재 시간 확인
        query5 = "참고로 현재 시간은 몇 시인가요?"
        print(f"사용자: {query5}")
        result5 = await agent.run(query5, thread=thread)
        print(f"에이전트: {result5.text}\n")

# 예제 실행
await travel_planning_agent()

print("\n" + "="*60)
print("워크샵을 완료했습니다! 🎉")
print("="*60)
print("\n다음 단계:")
print("1. 위 코드를 수정하여 다른 도구들을 추가해보세요")
print("2. 다른 클라이언트(Chat Client, Responses Client)로 변경해보세요")
print("3. 코드 인터프리터를 추가하여 데이터 분석 기능을 넣어보세요")
print("4. 이미지 분석 기능을 추가하여 여행지 사진을 분석해보세요")

=== 여행 계획 도우미 에이전트 ===

사용자: 도쿄로 여행을 가려고 하는데, 현재 날씨는 어떤가요?


[2025-11-03 11:00:18 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.
[2025-11-03 11:00:20 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.
[2025-11-03 11:00:20 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


에이전트: 현재 도쿄의 날씨는 폭풍입니다. 최고 기온은 11°C로, 다소 쌀쌀한 편입니다. 외출이나 여행 계획 시 우산과 방수 외투를 꼭 챙기시길 추천드립니다. 
추가로 궁금한 점이나 여행 준비에 도움이 필요하시면 언제든 말씀해 주세요!

사용자: 도쿄까지 항공권은 얼마나 할까요?


[2025-11-03 11:00:22 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.
[2025-11-03 11:00:25 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.
[2025-11-03 11:00:25 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


에이전트: 서울에서 도쿄까지의 왕복 항공권 예상 가격은 약 300,000원입니다. 항공사, 여행 시기, 예약 시점에 따라 금액이 달라질 수 있으니 참고해 주세요.

추가로 숙박이나 일일 여행 예산 계산이 필요하시면 말씀해 주세요!

사용자: 5일 동안 2명이 가는데, 일일 예산을 15만원으로 잡으면 총 얼마나 필요한가요?


...

에이전트: 2명이 5일 동안 도쿄로 여행을 갈 경우, 일일 예산을 15만 원으로 잡으면 총 여행 예산은 1,500,000원입니다.

여기에 항공권 약 300,000원(1인 기준)이 추가되면, 두 명 기준 총 예상 경비는 약 2,100,000원이 됩니다.

더 궁금한 점이나 세부 계획이 필요하시면 언제든 말씀해 주세요!

사용자: 지금까지 이야기한 내용을 바탕으로 총 여행 경비를 정리해주세요.


[2025-11-03 11:00:35 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


에이전트: 네, 지금까지 말씀하신 내용을 바탕으로 도쿄 여행에 필요한 총 경비를 정리해드리겠습니다.

---

1. 항공권 예상 가격  
- 서울 ↔ 도쿄 왕복 항공권: 약 300,000원 (1인 기준)  
- 2명 기준: 600,000원

2. 여행 예산  
- 여행 기간: 5일  
- 인원: 2명  
- 일일 예산: 150,000원  
- 총 여행 예산: 1,500,000원 (2명 × 5일 × 150,000원)

---

3. 도쿄 5일 2인 여행 예상 총 경비  
- 항공권: 600,000원  
- 여행 예산: 1,500,000원  
- **총합:** 2,100,000원

---

필요시 숙박, 입장료, 교통비, 기타 경비를 세부적으로 문의하실 수 있습니다.  
다른 도움이 필요하시면 언제든 말씀해 주세요!

사용자: 참고로 현재 시간은 몇 시인가요?


[2025-11-03 11:00:37 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


에이전트: 현재 UTC(세계 표준시) 시간은 2025년 11월 3일 02시 00분 37초입니다.

도쿄는 UTC+9 시간대를 사용하기 때문에, 현지 시간은 UTC 시간에 9시간을 더한 2025년 11월 3일 11시 00분 37초입니다.

더 궁금한 점 있으면 언제든 말씀해주세요!


워크샵을 완료했습니다! 🎉

다음 단계:
1. 위 코드를 수정하여 다른 도구들을 추가해보세요
2. 다른 클라이언트(Chat Client, Responses Client)로 변경해보세요
3. 코드 인터프리터를 추가하여 데이터 분석 기능을 넣어보세요
4. 이미지 분석 기능을 추가하여 여행지 사진을 분석해보세요


## 추가 학습 자료 및 참고사항

### 주요 개념 정리

1. **Azure OpenAI 클라이언트 유형**
   - **Chat Client**: 직접 채팅 기반 상호작용
   - **Responses Client**: 구조화된 응답 생성
   - **Assistants Client**: 영구 어시스턴트 및 자동 생명주기 관리

2. **함수 도구 (Function Tools)**
   - 에이전트 생성 시 정의: 모든 쿼리에서 사용 가능
   - 런 메서드에서 전달: 특정 쿼리에만 사용
   - 혼합 방식: 기본 도구 + 추가 도구

3. **스레드 관리**
   - 자동 스레드: 각 쿼리마다 새로운 컨텍스트
   - 명시적 스레드: 대화 컨텍스트 유지
   - 서비스 스레드 ID: 서버 측 대화 이력 저장

4. **코드 인터프리터**
   - Python 코드 실행 (Python 3.12+ 지원)
   - 수학적 계산 및 데이터 분석
   - 파일 업로드 및 처리

5. **멀티모달 기능**
   - 이미지 분석
   - 텍스트와 이미지 결합
   - 비전 모델 활용

### Python 3.12 환경 설정

```bash
# uv로 Python 3.12 환경 생성
uv venv --python 3.12

# 프로젝트 의존성 설치
uv sync --dev

# Python 버전 확인
uv run python --version  # Python 3.12.x

# Jupyter 노트북 실행
uv run jupyter notebook
```

### 환경 변수 설정 예제

```bash
# .env 파일
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME=gpt-4
AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME=gpt-4
AZURE_OPENAI_API_VERSION=2024-05-01-preview
```

### 인증 방법

1. **Azure CLI 인증** (권장)
   ```bash
   az login
   ```

2. **API 키 인증**
   ```python
   from openai import AsyncAzureOpenAI
   client = AsyncAzureOpenAI(api_key="your-api-key", ...)
   ```

3. **Managed Identity**
   ```python
   from azure.identity import DefaultAzureCredential
   credential = DefaultAzureCredential()
   ```

### 추가 리소스

- Microsoft Agent Framework 문서
- Azure OpenAI 서비스 문서
- GitHub 샘플 코드 저장소
- Python 3.12 공식 문서

### 다음 단계

1. 프로덕션 환경을 위한 에러 핸들링 추가
2. 로깅 및 모니터링 구현
3. 성능 최적화 및 캐싱 전략
4. 보안 모범 사례 적용
5. Python 3.12의 새로운 기능 활용
   - 개선된 타입 힌트
   - f-string 성능 향상
   - 새로운 pathlib 기능